In [ ]:
from pathlib import Path
import torch
import clip
from PIL import Image
import numpy as np
from pathlib import Path

In [ ]:
np.set_printoptions(suppress=True, precision=6) # limit sig figs of probabilities so we dont get something like 0.93817591827395717293
device = "cuda" if torch.cuda.is_available() else "cpu" # for now will always be cpu because I don't have a gpu

In [ ]:
#model, preprocess = clip.load("RN50", device=device) # Feel free to change model (use if through api)
local_dir = "./models/CLIP-RN50"
model_path = str(Path(local_dir) / "RN50.pt")
model, preprocess = clip.load(model_path, device=device) # If downloaded model directly

In [ ]:
directory = Path('./images')
images = torch.stack([preprocess(Image.open(p).convert("RGB")) for p in directory.glob('*')]).to(device) # just add images to images folder

In [ ]:
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device) # Retrieve text instead of hardcoding this, separate every sentence or two

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(images)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(images, text)
    probsText = logits_per_image.softmax(dim=-1).cpu().numpy()
    probsImage = logits_per_text.softmax(dim=-1).cpu().numpy()

In [ ]:
print("Label probs:", probsText)
print("Image probs:", probsImage)